# Apriori algorithm - consumer basket

In [1]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [17]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
from mlxtend.frequent_patterns import apriori, association_rules

In [19]:
df = pd.read_csv('../data/all_preprocessed.csv')
df.describe()

,author,content,poem name,age,type
count,506,506,505,506,506
unique,67,506,503,2,3
top,william shakespeare,let the bird of loudest layon the sole arabian...,Song,Renaissance,Love
freq,62,1,3,284,278


In [20]:
df['content'] = df['content'].apply(lambda x: x.split())
df['content']

0      [let, the, bird, of, loudest, layon, the, sole...
1      [sir, charles, into, my, chamber, coming, in, ...
2      [our, vice, runs, beyond, all, that, old, men,...
3      [lo, i, the, man, whose, muse, whilome, did, m...
4      [long, have, i, longd, to, see, my, love, agai...
                             ...                        
501    [this, youth, too, long, has, heard, the, brea...
502    [making, his, advanceshe, does, not, look, at,...
503    [i, thought, he, was, dumb, i, said, he, was, ...
504    [with, the, man, i, love, who, loves, me, not,...
505    [when, you, are, old, and, grey, and, full, of...
Name: content, Length: 506, dtype: object

In [21]:
stop_words = set(stopwords.words('english'))

In [22]:
df['content'] = df['content'].apply(lambda x: [item for item in x if item not in stop_words])
df['content']

0      [let, bird, loudest, layon, sole, arabian, tre...
1      [sir, charles, chamber, coming, writing, fairy...
2      [vice, runs, beyond, old, men, saw, far, authe...
3      [lo, man, whose, muse, whilome, maske, time, t...
4      [long, longd, see, love, againe, still, wisht,...
                             ...                        
501    [youth, long, heard, breakof, waters, land, ch...
502    [making, advanceshe, look, sniff, even, sniff,...
503    [thought, dumb, said, dumb, yet, ive, heard, c...
504    [man, love, loves, walked, street, lamps, flar...
505    [old, grey, full, sleep, nodding, fire, take, ...
Name: content, Length: 506, dtype: object

In [23]:
df['content']

0      [let, bird, loudest, layon, sole, arabian, tre...
1      [sir, charles, chamber, coming, writing, fairy...
2      [vice, runs, beyond, old, men, saw, far, authe...
3      [lo, man, whose, muse, whilome, maske, time, t...
4      [long, longd, see, love, againe, still, wisht,...
                             ...                        
501    [youth, long, heard, breakof, waters, land, ch...
502    [making, advanceshe, look, sniff, even, sniff,...
503    [thought, dumb, said, dumb, yet, ive, heard, c...
504    [man, love, loves, walked, street, lamps, flar...
505    [old, grey, full, sleep, nodding, fire, take, ...
Name: content, Length: 506, dtype: object

In [24]:
df['content'] = df['content'].apply(lambda x: list(set(x)))
df['content']

0      [simple, thus, sole, double, married, anthem, ...
1      [prayed, thin, thus, court, fames, mirth, mab,...
2      [laws, circes, extreme, safe, sits, excels, au...
3      [wight, sort, sprite, feend, grasping, liand, ...
4      [againe, bee, thus, still, sterne, full, fooli...
                             ...                        
501    [deserts, waitsstill, soil, anotherlook, leaps...
502    [mazy, awkward, reptiles, partial, skin, along...
503    [silencetearing, startling, spread, till, rabb...
504    [home, catch, flare, door, smile, saidthat, th...
505    [dream, sadly, mountains, full, many, slowly, ...
Name: content, Length: 506, dtype: object

In [25]:
newDF = pd.DataFrame(columns =['document', 'word'])
for i, content in enumerate(df.content):
    doc = [i] * len(content)
    
    tmp = pd.DataFrame(list(zip(doc, content)), columns =['document', 'word'])
    
    newDF = pd.concat([newDF, tmp])
newDF

,document,word
0,0,simple
1,0,thus
2,0,sole
3,0,double
4,0,married
...,...,...
43,505,deep
44,505,love
45,505,amid
46,505,moments


Sorting fo apriori algorithm

In [26]:
sortedDf = newDF.sort_values(by=['document', 'word'])
sortedDf

,document,word
5,0,anthem
102,0,appalledthat
31,0,arabian
37,0,asunder
153,0,augur
...,...,...
10,505,soul
21,505,stars
11,505,take
29,505,true


In [27]:
# sortedDf.to_csv('data.csv', index=True)

In [28]:
basket = pd.crosstab(index=sortedDf['document'], columns=sortedDf['word'])
basket

word,1,10,13,1538,19,1904,1914,1917,1919,1921,...,yvory,ywis,zeal,zealous,zenophontes,zephyrs,zephyrus,zeus,zone,zoo
document,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
502,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
503,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
frequentItemsets = apriori(basket, min_support=0.05, use_colnames=True)
frequentItemsets

,support,itemsets
0,0.057312,(air)
1,0.065217,(alas)
2,0.053360,(alone)
3,0.051383,(another)
4,0.086957,(art)
...,...,...
484,0.051383,"(thou, thee, yet)"
485,0.053360,"(thee, thy, yet)"
486,0.057312,"(thou, thine, thy)"
487,0.065217,"(thou, thy, yet)"


In [30]:
rules = association_rules(frequentItemsets, metric="confidence", min_threshold=0.8)

In [31]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(poems),(permission),0.114625,0.154150,0.112648,0.982759,6.375332,0.094979,49.059289,0.952303
1,(publishing),(permission),0.059289,0.154150,0.059289,1.000000,6.487179,0.050149,inf,0.899160
2,(reprinted),(permission),0.086957,0.154150,0.086957,1.000000,6.487179,0.073552,inf,0.926407
3,(used),(permission),0.071146,0.154150,0.065217,0.916667,5.946581,0.054250,10.150198,0.895551
4,(used),(poems),0.071146,0.114625,0.061265,0.861111,7.512452,0.053110,6.374704,0.933288
5,(thee),(thy),0.154150,0.209486,0.124506,0.807692,3.855588,0.092214,4.110672,0.875612
6,(thine),(thou),0.077075,0.193676,0.067194,0.871795,4.501308,0.052266,6.289328,0.842801
7,(thine),(thy),0.077075,0.209486,0.063241,0.820513,3.916788,0.047095,4.404291,0.806879
8,"(thou, art)",(thy),0.067194,0.209486,0.059289,0.882353,4.211987,0.045212,6.719368,0.817514
9,"(art, thy)",(thou),0.061265,0.193676,0.059289,0.967742,4.996708,0.047423,24.996047,0.852070
